In [1]:
import sys
import os
import json
import numpy as np

sys.path.append('..')
####
# sys.path.append('../../../../tests')
# import atelier_import_spoofer
###
import fake_lightgbm as lgb
from pipeline import pipeline

"atelier_fabrique" ->  "/home/sk/opencore/atelier/tests/../atelier_fabrique/__init__.py" spoofed


In [6]:
samp_dir = './samples'
filenames = [f for f in os.listdir(samp_dir) if os.path.isfile(f'{samp_dir}/{f}')]
samples = []
for filename in filenames:
    with open(f'{samp_dir}/{filename}') as fp:
        samples.append(fp.read())


In [3]:
pipeline.start_emulation(sample_gen)

pipeline.json is generated in /home/sk/opencore/atelier/atelier_fabrique/examples/fake_lightgbm
10.166.48.4:52525/python:3.6-centos-krb5-43
Dockerfile is generated in /home/sk/opencore/atelier/atelier_fabrique/examples/fake_lightgbm
extractor started
extractor stopped
model started
model stopped
metric_collector started
metric_collector stopped


In [4]:
# load model
bst = lgb.Booster(model_file='../model.txt')  # init model
classes = ["about", "services", "contacts", "roaming", "tariffs",
           "simcards", "balance", "internet", "messaging", "support"]

In [7]:
# simple inference code
reference_results = []  # we will use this results for tests
for sample in samples:
    ## 1. extract features from json message
    # 1.1. parse message and get features
    mes = json.loads(sample)
    features = np.array([mes['traffic'][cls] for cls in classes]).flatten()
    ## 2. make prediction result
    # 2.1 get prediction
    pred_vals = bst.predict([features, ])[0]
    # 2.2 normalize scores, get class
    try:
        scores = pred_vals / pred_vals.sum()
    except:
        scores = pred_vals
    reason = classes[scores.argmax()]
    # 2.3 make and serialize message
    scores_dict = {cls: round(scores[i], 2) for i, cls in enumerate(classes)}
    res = dict(ts=mes['ts'], uid=mes['uid'], number=mes['number'], classes=scores_dict, reason=reason)
    reference_results.append(json.dumps(res))

In [8]:
# load results of emulation
result_dir = './out/data'
filenames = [f for f in os.listdir(result_dir) if os.path.isfile(f'{result_dir}/{f}')]
results = []
for filename in filenames:
    with open(f'{result_dir}/{filename}') as fp:
        results.append(fp.read())

In [9]:
#check if results are equal
assert set(results) == set(reference_results)